# Machine Translation (English to Hindi)

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential, Model,save_model,load_model
from keras.layers import LSTM, Dense, Bidirectional,Input

import os

# Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/MachineTransalation')

In [ ]:
with open('DATASET.txt','r',encoding='utf-8') as f:
    lines = f.read().split('\n')

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to traina for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "DATASET.txt"

In [ ]:
range(min(num_samples, len(lines) - 1))

range(0, 2867)

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [ ]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)



In [ ]:
len(input_characters) , len(target_characters)

(70, 92)

In [ ]:
input_texts[:10]

['Wow!',
 'Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Got it?']

In [ ]:
target_texts[:10]

['\tवाह!\n',
 '\tबचाओ!\n',
 '\tउछलो.\n',
 '\tकूदो.\n',
 '\tछलांग.\n',
 '\tनमस्ते।\n',
 '\tनमस्कार।\n',
 '\tवाह-वाह!\n',
 '\tचियर्स!\n',
 '\tसमझे कि नहीं?\n']

In [ ]:
len(input_character)

70

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 2867
Number of unique input tokens: 70
Number of unique output tokens: 92
Max sequence length for inputs: 124
Max sequence length for outputs: 123


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# Decalaring the dimensions

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

In [ ]:
encoder_input_data.shape


(2867, 124, 70)

In [ ]:
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [ ]:
decoder_input_data.shape

(2867, 123, 92)

In [ ]:

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [ ]:
decoder_target_data.shape

(2867, 123, 92)

# One Hot Representation

In [ ]:

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
        encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


# Model Building

### Encoder

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
import keras

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

es = EarlyStopping(monitor='val_loss',mode='min',verbose=1, patience=200)
mc = ModelCheckpoint('hinditoenglishmodel.h5',monitor='val_accuracy', mode='max', save_best_only=True,verbose=1)


In [ ]:
## Train the model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
history =model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=1000,
    validation_split=0.2,
    callbacks=[es,mc])
# Save model

Epoch 1/1000
36/36 [==============================] - ETA: 0s - loss: 1.3116 - accuracy: 0.7844
Epoch 1: val_accuracy improved from -inf to 0.68070, saving model to hinditoenglishmodel.h5
36/36 [==============================] - 12s 107ms/step - loss: 1.3116 - accuracy: 0.7844 - val_loss: 1.7285 - val_accuracy: 0.6807
Epoch 2/1000
36/36 [==============================] - ETA: 0s - loss: 0.8880 - accuracy: 0.8072
Epoch 2: val_accuracy improved from 0.68070 to 0.68784, saving model to hinditoenglishmodel.h5
36/36 [==============================] - 1s 33ms/step - loss: 0.8880 - accuracy: 0.8072 - val_loss: 2.1520 - val_accuracy: 0.6878
Epoch 3/1000
34/36 [===========================>..] - ETA: 0s - loss: 0.8458 - accuracy: 0.8086
Epoch 3: val_accuracy improved from 0.68784 to 0.69001, saving model to hinditoenglishmodel.h5
36/36 [==============================] - 1s 30ms/step - loss: 0.8444 - accuracy: 0.8087 - val_loss: 1.5113 - val_accuracy: 0.6900
Epoch 4/1000
36/36 [==================

In [ ]:
import keras

In [ ]:
model =keras.models.load_model("hinditoenglishmodel.h5")

## Inference Setup

In [ ]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="inpu t_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

## Generating the Decoded Sentence for given input sentence

In [ ]:
seq_index=23
inp_seq = encoder_input_data[seq_index:seq_index+1]
translated_sent = decode_sequence(inp_seq)
print('-')
print('Input sentence:', input_texts[seq_index])
print('Decoded sentence:', translated_sent)

1/1 [==============================] - 0s 18ms/step
-
Input sentence: I forgot.
Decoded sentence: मुझे उसके पास में का क्या है?



In [ ]:
for seq_index in range(5):
    inp_seq = encoder_input_data[seq_index:seq_index+1]
    translated_sent = decode_sequence(inp_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', translated_sent)

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Wow!
Decoded sentence: मुझे उसके पास में का क्या है?

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Help!
Decoded sentence: मुझे उसके पास में का क्या है?

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Jump.
Decoded sentence: मुझे उसके पास में का क्या है?

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Jump.
Decoded sentence: मुझे उसके पास में का क्या है?

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Jump.
Decoded sentence: मुझे उसके पास में का क्या है?

